<h1>In the tech and AI world, 
CNN stands for Convolutional Neural Network — a deep learning 
algorithm primarily used in image recognition and computer vision tasks. Key Points: Purpose: Processes and recognizes patterns in visual data</h1>

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import glob

# 🔹 Dataset path
dataset_path = r"F:\All My Website using Next.js code\Machine Language\Data Science\CNN"

print("🔍 Checking dataset structure...")
print(f"Dataset path: {dataset_path}")

# 🔹 Function to check and display folder structure
def check_dataset_structure(path):
    """Check and display the dataset folder structure"""
    if not os.path.exists(path):
        print(f"❌ Path does not exist: {path}")
        return False, None
    
    print(f"✅ Path exists: {path}")
    
    # List all subdirectories
    subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    print(f"📁 Found subdirectories: {subdirs}")
    
    # Count images in each subdirectory
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    class_counts = {}
    total_images = 0
    
    for subdir in subdirs:
        subdir_path = os.path.join(path, subdir)
        image_count = 0
        
        for ext in image_extensions:
            pattern = os.path.join(subdir_path, f"*{ext}")
            image_count += len(glob.glob(pattern))
            pattern = os.path.join(subdir_path, f"*{ext.upper()}")
            image_count += len(glob.glob(pattern))
        
        class_counts[subdir] = image_count
        total_images += image_count
        print(f"  📸 {subdir}: {image_count} images")
    
    print(f"🔢 Total images found: {total_images}")
    
    if total_images == 0:
        print("❌ No images found in any subdirectory!")
        return False, None
    
    return True, class_counts

# 🔹 Check the dataset
has_data, class_counts = check_dataset_structure(dataset_path)

if not has_data:
    print("\n🚨 DATASET ISSUE DETECTED!")
    print("\n📋 Expected folder structure:")
    print("CNN/")
    print("├── dog/")
    print("│   ├── image1.jpg")
    print("│   ├── image2.jpg")
    print("│   └── ...")
    print("├── cat/")
    print("│   ├── image1.jpg")
    print("│   └── ...")
    print("└── lion/")
    print("    ├── image1.jpg")
    print("    └── ...")
    print("\n💡 Make sure:")
    print("1. Your images are in the correct folders")
    print("2. Images have valid extensions (.jpg, .jpeg, .png, .bmp, .gif)")
    print("3. Folder names match your class names")
    exit()

# 🔹 Create sample dataset if needed
def create_sample_dataset():
    """Create a sample dataset for testing"""
    print("\n🔧 Would you like to create a sample dataset for testing? (y/n)")
    response = input().lower().strip()
    
    if response == 'y':
        try:
            import numpy as np
            from PIL import Image
            
            # Create sample images
            classes = ['dog', 'cat', 'lion']
            samples_per_class = 50
            
            for class_name in classes:
                class_dir = os.path.join(dataset_path, class_name)
                os.makedirs(class_dir, exist_ok=True)
                
                for i in range(samples_per_class):
                    # Create random colored image
                    img_array = np.random.randint(0, 256, (128, 128, 3), dtype=np.uint8)
                    img = Image.fromarray(img_array)
                    img.save(os.path.join(class_dir, f'{class_name}_{i:03d}.jpg'))
                
                print(f"✅ Created {samples_per_class} sample images for {class_name}")
            
            print("🎉 Sample dataset created successfully!")
            return True
            
        except ImportError:
            print("❌ PIL not installed. Install it with: pip install Pillow")
            return False
        except Exception as e:
            print(f"❌ Error creating sample dataset: {e}")
            return False
    
    return False

# If no data found, offer to create sample dataset
if sum(class_counts.values()) < 10:  # Less than 10 total images
    print("⚠️ Very few images found. This might not be enough for training.")
    if create_sample_dataset():
        # Recheck after creating sample data
        has_data, class_counts = check_dataset_structure(dataset_path)

# 🔹 CNN Model (simplified for testing)
print("\n🏗️ Building CNN model...")
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(class_counts), activation='softmax')  # Dynamic number of classes
])

cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("📊 Model summary:")
cnn.summary()

# 🔹 Create data generators with extensive debugging
print("\n📊 Creating data generators...")

# Training data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# 🔹 Try to create generators with error handling
try:
    print("🔄 Creating training generator...")
    train_set = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(64, 64),
        batch_size=16,  # Smaller batch size for testing
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    print(f"✅ Training generator created successfully!")
    print(f"   📸 Training samples: {train_set.samples}")
    print(f"   📁 Classes found: {list(train_set.class_indices.keys())}")
    print(f"   🎯 Class indices: {train_set.class_indices}")
    
    print("\n🔄 Creating validation generator...")
    val_set = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    print(f"✅ Validation generator created successfully!")
    print(f"   📸 Validation samples: {val_set.samples}")
    
    # Check if generators have data
    if train_set.samples == 0:
        raise ValueError("Training generator has 0 samples!")
    if val_set.samples == 0:
        raise ValueError("Validation generator has 0 samples!")
    
except Exception as e:
    print(f"❌ Error creating data generators: {e}")
    print("\n🔍 Debugging information:")
    print(f"Directory contents: {os.listdir(dataset_path)}")
    
    # Try without validation split
    print("\n🔄 Trying without validation split...")
    try:
        simple_datagen = ImageDataGenerator(rescale=1./255)
        simple_generator = simple_datagen.flow_from_directory(
            dataset_path,
            target_size=(64, 64),
            batch_size=16,
            class_mode='categorical',
            shuffle=True
        )
        print(f"✅ Simple generator works! Found {simple_generator.samples} samples")
        
        # Use simple generator for both train and val (not ideal but for testing)
        train_set = simple_generator
        val_set = simple_generator
        
    except Exception as e2:
        print(f"❌ Simple generator also failed: {e2}")
        exit()

# 🔹 Test the generators by getting a batch
print("\n🧪 Testing generators...")
try:
    print("Getting a batch from training generator...")
    batch_x, batch_y = next(train_set)
    print(f"✅ Successfully got batch: X shape {batch_x.shape}, Y shape {batch_y.shape}")
    print(f"   🎯 Batch size: {len(batch_x)}")
    print(f"   📊 X range: [{batch_x.min():.3f}, {batch_x.max():.3f}]")
    print(f"   📊 Y sample: {batch_y[0]}")
    
except Exception as e:
    print(f"❌ Error getting batch: {e}")
    exit()

# 🔹 Train the model
print("\n🚀 Starting training...")
try:
    # Use fewer epochs for initial testing
    history = cnn.fit(
        train_set,
        epochs=5,  # Start with fewer epochs
        validation_data=val_set,
        verbose=1
    )
    
    print("\n🎉 Training completed successfully!")
    print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")
    
    # Save the model
    model_path = os.path.join(dataset_path, 'cnn_model.h5')
    cnn.save(model_path)
    print(f"💾 Model saved to: {model_path}")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    print("🔍 This might be due to:")
    print("1. Insufficient data")
    print("2. Memory issues")
    print("3. Incompatible image formats")

# 🔹 Quick prediction test
print("\n🔮 Testing prediction on a sample...")
try:
    # Get a sample from the generator
    test_batch_x, test_batch_y = next(train_set)
    prediction = cnn.predict(test_batch_x[:1])  # Predict on first image
    predicted_class = list(train_set.class_indices.keys())[prediction.argmax()]
    actual_class = list(train_set.class_indices.keys())[test_batch_y[0].argmax()]
    
    print(f"✅ Prediction test successful!")
    print(f"   🎯 Predicted: {predicted_class} (confidence: {prediction.max():.3f})")
    print(f"   ✅ Actual: {actual_class}")
    
except Exception as e:
    print(f"❌ Prediction test failed: {e}")

print("\n✨ Script completed!")

🔍 Checking dataset structure...
Dataset path: F:\All My Website using Next.js code\Machine Language\Data Science\CNN
✅ Path exists: F:\All My Website using Next.js code\Machine Language\Data Science\CNN
📁 Found subdirectories: ['cat', 'dog', 'lion']
  📸 cat: 4 images
  📸 dog: 4 images
  📸 lion: 4 images
🔢 Total images found: 12

🏗️ Building CNN model...
📊 Model summary:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,454,595 (9.36 MB)

 Trainable params: 2,454,595 (9.36 MB)

 Non-trainable params: 0 (0.00 B)


📊 Creating data generators...
🔄 Creating training generator...
Found 6 images belonging to 3 classes.
✅ Training generator created successfully!
   📸 Training samples: 6
   📁 Classes found: ['cat', 'dog', 'lion']
   🎯 Class indices: {'cat': 0, 'dog': 1, 'lion': 2}

🔄 Creating validation generator...
Found 0 images belonging to 3 classes.
✅ Validation generator created successfully!
   📸 Validation samples: 0
❌ Error creating data generators: Validation generator has 0 samples!

🔍 Debugging information:
Directory contents: ['cat', 'dog', 'lion']

🔄 Trying without validation split...
Found 6 images belonging to 3 classes.
✅ Simple generator works! Found 6 samples

🧪 Testing generators...
Getting a batch from training generator...
✅ Successfully got batch: X shape (6, 64, 64, 3), Y shape (6, 3)
   🎯 Batch size: 6
   📊 X range: [0.000, 1.000]
   📊 Y sample: [0. 1. 0.]

🚀 Starting training...
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1667 - loss: 1.0927WARNING:tensorflow:


Final training accuracy: 0.3333
Final validation accuracy: 0.6667
💾 Model saved to: F:\All My Website using Next.js code\Machine Language\Data Science\CNN\cnn_model.h5

🔮 Testing prediction on a sample...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
✅ Prediction test successful!
   🎯 Predicted: dog (confidence: 0.507)
   ✅ Actual: dog

✨ Script completed!
